<a href="https://colab.research.google.com/github/Neniflight/fa22-reddit-politics-gain/blob/main/ACM_AI_FA22_Team_2_Reddit_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ACM AI Project Team 2 - GAIN

### Classification of Political Biases in Reddit Posts Using RNN

### Cloning dataset 

In [ ]:
!git clone https://github.com/Neniflight/fa22-reddit-politics-gain

Cloning into 'fa22-reddit-politics-gain'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [ ]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

import time

from torch.utils.data import DataLoader
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from sklearn.model_selection import train_test_split
import torchtext
import random
torch.backends.cudnn.deterministic = True

In [ ]:
def full_text(df):
  title = df.get('Title')
  text = df.get('Text')
  all_text = []
  for i in range(df.shape[0]):
    if type(text[i]) == float:
      new_text = title[i].replace("\n", "")
    else:
      new_text = title[i].replace("\n", "") + " " + text[i].replace("\n", "")
    all_text.append(new_text)
  return all_text

In [ ]:
df = pd.read_csv('fa22-reddit-politics-gain/liberal_vs_conservative.csv')
df

,Title,Political Lean,Score,Id,Subreddit,URL,Num of Comments,Text,Date Created
0,"No matter who someone is, how they look like, ...",Liberal,1,t5fybt,socialism,https://v.redd.it/ng5fyl7hp2l81,0,NaN,1646271777
1,Biden speech draws 38.2 million U.S. TV viewers,Liberal,6,t5fqdn,democrats,https://www.reuters.com/world/us/biden-speech-...,1,NaN,1646271108
2,State of the union,Liberal,1,t5fj9a,DemocraticSocialism,https://www.reddit.com/r/DemocraticSocialism/c...,1,Who watched the state of the union last night ...,1646270488
3,We Should Just Give Poor People Money,Liberal,7,t5f7n9,SocialDemocracy,https://youtu.be/a80kRjpubG0,3,NaN,1646269508
4,Do it for the Dew,Liberal,6,t5es2c,democrats,https://i.redd.it/drmunn90f2l81.jpg,1,NaN,1646268183
...,...,...,...,...,...,...,...,...,...
12849,Ron Paul’s Spirited Defense of WikiLeaks & Fre...,Conservative,2,em7rm,anarchocapitalism,http://www.deathandtaxesmag.com/40485/ron-paul...,1,NaN,1292421269
12850,"“Anarcho-capitalism, in my opinion, is a doctr...",Conservative,2,ei98o,anarchocapitalism,http://www.pressaction.com/news/weblog/full_ar...,1,NaN,1291812794
12851,Mises Wiki is a wiki project dedicated to the ...,Conservative,2,e6x22,anarchocapitalism,http://wiki.mises.org/wiki/Main_Page,0,NaN,1289909847
12852,Fireman Protection Monopoly - Is This Failed C...,Conservative,2,e4vtd,anarchocapitalism,http://www.csmonitor.com/Business/Mises-Econom...,0,NaN,1289523634


In [ ]:
df['full_text'] = full_text(df)
df['full_text'] = df['full_text'].astype('str')

In [ ]:
df = df[['Political Lean', 'full_text']]
df['Political Lean'] = df['Political Lean'].replace(["Liberal", "Conservative"], [0, 1])
df = df.rename(columns = {"Political Lean": "label", "full_text": "text"})
df

<ipython-input-9-f58e51ebf8ff>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Political Lean'] = df['Political Lean'].replace(["Liberal", "Conservative"], [0, 1])


,label,text
0,0,"No matter who someone is, how they look like, ..."
1,0,Biden speech draws 38.2 million U.S. TV viewers
2,0,State of the union Who watched the state of th...
3,0,We Should Just Give Poor People Money
4,0,Do it for the Dew
...,...,...
12849,1,Ron Paul’s Spirited Defense of WikiLeaks & Fre...
12850,1,"“Anarcho-capitalism, in my opinion, is a doctr..."
12851,1,Mises Wiki is a wiki project dedicated to the ...
12852,1,Fireman Protection Monopoly - Is This Failed C...


In [ ]:
nice = [len(i.split()) for i in df['text']]
df['count'] = nice
df = df[(df['count'] >= 5) & (df['count'] <= 100)]
df = df[['label', 'text']]
df = df.reset_index(drop = True)

In [ ]:
df

,label,text
0,0,"No matter who someone is, how they look like, ..."
1,0,Biden speech draws 38.2 million U.S. TV viewers
2,0,State of the union Who watched the state of th...
3,0,We Should Just Give Poor People Money
4,0,Do it for the Dew
...,...,...
11020,1,Ron Paul’s Spirited Defense of WikiLeaks & Fre...
11021,1,"“Anarcho-capitalism, in my opinion, is a doctr..."
11022,1,Mises Wiki is a wiki project dedicated to the ...
11023,1,Fireman Protection Monopoly - Is This Failed C...


In [ ]:
#Loading in data
df_conservative, df_liberal = df[df.get("label") == 1], df[df.get("label") == 0]
df_con, df_test_con = train_test_split(df_conservative, train_size = 0.9)
df_lib, df_test_lib = train_test_split(df_liberal, train_size = 0.9)
df_test = pd.concat([df_test_lib, df_test_con], ignore_index = True)

In [ ]:
df_train = pd.concat([df_lib, df_con], ignore_index = True)
df_train

,label,text
0,0,"They're already far in the ""negative credit"" zone"
1,0,"What U.S. abortion access looks like, in graphics"
2,0,The effective federal corporate income tax rat...
3,0,EROL's incredible resource on previous communi...
4,0,Meet Tom Suozzi - The Democrat leading the cha...
...,...,...
9916,1,WATCH: DeSantis ad roasts Jen Psaki | National...
9917,1,It is one thing for the police to use force to...
9918,1,So I posed a question over in antiwork about h...
9919,1,The Death Penalty feat. PragerU


In [ ]:
df_test

,label,text
0,0,School District Declines to Remove Michelle Ob...
1,0,Harris was inside DNC on Jan. 6 when pipe bomb...
2,0,"In the past 20 years, Republican victory margi..."
3,0,“This is a CRUEL way to run a welfare system” ...
4,0,"Talking Union at Recreational Equipment, Inc. ..."
...,...,...
1099,1,Jan. 6 committee accused of violating House ru...
1100,1,The fact r/antiwork is full of communists goes...
1101,1,Two Manhattan prosecutors leading criminal tax...
1102,1,"Kyiv’s Chief Rabbi and Family, Stay Put to Hel..."


In [ ]:
train_dataset = list(df_train.to_records(index=False))
test_dataset  = list(df_test.to_records(index=False))

# test_dataset  = [(0 if label in range(5) else 1, text) 
#                  for label, text in test_dataset if label != 5]

In [ ]:
tokenizer = get_tokenizer('basic_english')

def build_vocab(dataset) :
  for _, text in dataset :
    yield tokenizer(text)

vocab = build_vocab_from_iterator(build_vocab(train_dataset), 
                                  min_freq=1, specials=['<UNK>'])
vocab.set_default_index(vocab['<UNK>'])

In [ ]:
max_words = 100
def vectorize_batch(batch) :

  Y, X = list(zip(*batch))
  X = [vocab(tokenizer(text)) for text in X]
  X = [tokens + ([0] * (max_words - len(tokens))) 
       if len(tokens) < max_words else tokens[:max_words]
       for tokens in X]

  return torch.tensor(X, dtype=torch.int32), torch.tensor(Y)

train_dataloader = DataLoader(train_dataset, batch_size=128, 
                              collate_fn=vectorize_batch, shuffle=True)
test_dataloader  = DataLoader(test_dataset, batch_size=128, 
                              collate_fn=vectorize_batch)

In [ ]:
for i, (text, label) in enumerate(train_dataloader) :
  print(text.shape)
  print(label.shape)
  break

torch.Size([128, 100])
torch.Size([128])


In [ ]:
class RNN(nn.Module) :

  def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim) :

    super().__init__()

    self.embedding = nn.Embedding(input_dim, embedding_dim)
    self.rnn = nn.RNN(embedding_dim, hidden_dim, nonlinearity='relu', batch_first=True)
    # self.rnn = torch.nn.LSTM(embedding_dim, hidden_dim)
    self.fc = nn.Linear(hidden_dim, output_dim)

  def forward(self, text) :

    embedded = self.embedding(text)

    output, hidden = self.rnn(embedded)
    hidden.squeeze_(0)
    output = self.fc(hidden)

    return output

In [ ]:
random_seed, learning_rate, num_classes = (12, 0.001, 2)
embedding_dim, hidden_dim = (128, 256)

torch.manual_seed(random_seed)

model = RNN(input_dim= vocab.__len__(),
            embedding_dim=embedding_dim,
            hidden_dim=hidden_dim,
            output_dim=num_classes)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def compute_accuracy(model, dataloader) :

  with torch.no_grad() :

    correct_pred, num_examples = 0, 0

    for i, (features, targets) in enumerate(dataloader) :

      features = features
      targets = targets.float()

      logits = model(features)
      _, predicted_labels = torch.max(logits, 1)

      num_examples += targets.size(0)
      correct_pred += (predicted_labels == targets).sum()
  
  return correct_pred.float()/num_examples * 100

In [ ]:
start_time = time.time()
NUM_EPOCHS = 10
train_valid_accuracy = []
losses = []

for epoch in range(NUM_EPOCHS) :

  model.train()

  for batch_idx, batch_data in enumerate(train_dataloader) :

    text = batch_data[0]
    labels = batch_data[1]
    
    # Forward Propagation
    logits = model(text)

    # Clear Gradients
    optimizer.zero_grad()

    # Calculate Cross Entropy Loss
    loss = F.cross_entropy(logits, labels)

    # Calculating Gradients
    loss.backward()

    # Update Model Parameters
    optimizer.step()

    if not batch_idx % 20 :
      losses.append(loss.item())
      print(f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
            f'Batch: {batch_idx:03d}/{len(train_dataloader):03d} | '
            f'Loss: {loss:.4f}')
      
  with torch.set_grad_enabled(False) :
    train_accuracy = compute_accuracy(model, train_dataloader)
    # test_accuracy = compute_accuracy(model, test_dataloader)
    train_valid_accuracy.append([train_accuracy.item()])
    print(f'training accuracy: '
          f'{train_accuracy:.2f}%')
          # f'\ntest accuracy: '
          # f'{test_accuracy:.2f}%')
      
  print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')

print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')

Epoch: 001/010 | Batch: 000/078 | Loss: 0.6415
Epoch: 001/010 | Batch: 020/078 | Loss: 0.6318
Epoch: 001/010 | Batch: 040/078 | Loss: 0.6708
Epoch: 001/010 | Batch: 060/078 | Loss: 0.6241
training accuracy: 65.67%
valid accuracy: 65.22%
Time elapsed: 0.55 min
Epoch: 002/010 | Batch: 000/078 | Loss: 0.6104
Epoch: 002/010 | Batch: 020/078 | Loss: 0.6430
Epoch: 002/010 | Batch: 040/078 | Loss: 0.6742
Epoch: 002/010 | Batch: 060/078 | Loss: 0.6404
training accuracy: 65.64%
valid accuracy: 65.40%
Time elapsed: 1.11 min
Epoch: 003/010 | Batch: 000/078 | Loss: 0.6203
Epoch: 003/010 | Batch: 020/078 | Loss: 0.6419
Epoch: 003/010 | Batch: 040/078 | Loss: 0.5785
Epoch: 003/010 | Batch: 060/078 | Loss: 0.6791
training accuracy: 65.66%
valid accuracy: 65.13%
Time elapsed: 1.71 min
Epoch: 004/010 | Batch: 000/078 | Loss: 0.6171


KeyboardInterrupt: ignored

In [ ]:
with torch.set_grad_enabled(False) :
    print(f'test accuracy: '
          f'{compute_accuracy(model, test_dataloader):.2f}%') 

test accuracy: 65.13%
